1. Get data (yfinance/api)
2. find mean, stdev and correlation
3. create table for weightage
4. create MVE portfolio (solver) 
5. plot MVE line with CAL line
6. 

In [ ]:
# Import Library
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import os


In [42]:
# Create a matrix of stock return and standard deviation
asset_risk_free = 0.05

asset_risk_return_dict={"Return":{"US":0.1355, "UK":0.1589,"France":0.1519,"Germany":0.1435,"Japan":0.1497},
                        "Deviation":{"US":0.1535, "UK":0.2430,"France":0.2324,"Germany":0.2038,"Japan":0.2298},

                        }


asset_correlation_matrix_dict={"US":{"US":1,"UK":0.5003,"France":0.4398,"Germany":0.3681,"Japan":0.2663},
                          "UK":{"US":0.5003,"UK":1,"France":0.5420,"Germany":0.4265,"Japan":0.3581},
                          "France":{"US":0.4398,"UK":0.5420,"France":1,"Germany":0.6032,"Japan":0.3923},
                          "Germany":{"US":0.3681,"UK":0.4265,"France":0.6032,"Germany":1,"Japan":0.3663},
                          "Japan":{"US":0.2663,"UK":0.3581,"France":0.3923,"Germany":0.3663,"Japan":1}}


asset_risk_return= pd.DataFrame(asset_risk_return_dict)
asset_correlation_matrix= pd.DataFrame(asset_correlation_matrix_dict)


# Create weight range from 0 to 1 (inclusive) with 0.1 intervals
w_us = np.round(np.arange(0, 1.1, 0.1), 3)
w_japan = np.round(1 - w_us, 3)

# Create DataFrame with weight range
portfolio_weights = pd.DataFrame({
    "w1": w_us,
    "w2": w_japan
})


# Create a range of return based on weightage
portfolio_return_list=[]
for index,row in portfolio_weights.iterrows():
    us_return= row["w1"]*asset_risk_return["Return"]["US"]
    jp_return= row["w2"]*asset_risk_return["Return"]["Japan"]
    weighted_portfolio_return= us_return+jp_return
    portfolio_return_list.append(weighted_portfolio_return)

# Create Dataframe of weighted return
portfolio_mean=pd.DataFrame({
    "return":portfolio_return_list
})

# Calculate Variance for each weight
portfolio_variance_list=[]
for index, row in portfolio_weights.iterrows():
    us_volatility= (row["w1"])**2 *asset_risk_return["Deviation"]["US"] **2
    jp_volatility= (row["w2"])**2 *asset_risk_return["Deviation"]["Japan"] **2
    us_jp_deviation= 2*(row["w1"])*(row["w2"])*asset_risk_return["Deviation"]["US"]*asset_risk_return["Deviation"]["Japan"] *asset_correlation_matrix["US"]["Japan"]
    weighted_risk=us_volatility+jp_volatility+us_jp_deviation
    portfolio_variance_list.append(weighted_risk)

# Calculate Standard Deviation for each weight
portfolio_stdev_list=[]
for var in portfolio_variance_list:
    portfolio_stdev_list.append((np.sqrt(var)))

# Calculate Volatility for each weight
portfolio_volatility=pd.DataFrame({
    "variance":portfolio_variance_list,
    "stdev":portfolio_stdev_list
})

# Calculate Sharpe Ratio for each weight
portfolio_sharpe_list=[]
for idx, mean in enumerate(portfolio_return_list):
    sharpe= (mean-asset_risk_free)/portfolio_stdev_list[idx]
    portfolio_sharpe_list.append(sharpe)

capital_allocation_matrix=pd.DataFrame({
    "w(US)":w_us,
    "w(Japan)":w_japan,
    "Mean Return":portfolio_return_list,
    "Variance":portfolio_variance_list,
    "Standard Deviation": portfolio_stdev_list,
    "Sharpe Ratio": portfolio_sharpe_list
})